# Adding Latitude and Longitude Coordinates

In [1]:
from pathlib import Path

import geonamescache
import pandas as pd
from unidecode import unidecode

## Load previously parsed headlines from parquet file

In [2]:
parsed_headlines = pd.read_parquet(Path("data/processed/parsed-headlines.parquet"))
parsed_headlines.describe()

,headline,city,country
count,608,608,15
unique,608,576,10
top,Zika arrives in San Onofre,Miami,Malaysia
freq,1,4,3


## Prepare parsed headlines to merge with `geonamescache` table

Most headlines did not contain countries that could be parsed. Lets drop the column entirely and use `geonamescache` to find the country code from the parsed cities. We are also adding an extra column called `city_lower` which is the lowercase version of the city names. This will be used to help merge data from the `geonamescache` library to this table.

In [3]:
parsed_headlines.drop(["country"], axis=1, inplace=True, errors="ignore")
parsed_headlines["city_lower"] = parsed_headlines.city.apply(str.lower)
parsed_headlines

,headline,city,city_lower
0,Case of Hepatitis A Reported in Calgary,Calgary,calgary
1,More people in Nadi are infected with HIV ever...,Nadi,nadi
2,Lower Hospitalization in Janesville after Vari...,Janesville,janesville
3,Contaminated Meat Brings Trouble for Bonn Farmers,Bonn,bonn
4,Molo Cholera Spread Causing Concern,Molo,molo
...,...,...,...
603,Lower Hospitalization in Lakewood after Hepati...,Lakewood,lakewood
604,More Patients in Canton are Getting Diagnosed ...,Canton,canton
605,Zika case confirmed in Lorain,Lorain,lorain
606,Hepatitis E Hits Hammond,Hammond,hammond


## Explore the available city data in `geonamescache`

Lets take a look at all the available city names that are returned by the `geonamescache.get_cities` method. We normalize the city name using `unidecode`, and then shove everything into a `pandas.DataFrame` and get the statistics from there.

It looks like there are 24336 cities returned by the `get_cities` method, but after normalizing with `unidecode` there are only 23022 unique city names. Unique city names can be merged directly with the headlines `DataFrame`. However, if the city name is not unique, we will need to choose one city. The selectrion criteria is to choose the city with the largest population.

In [4]:
gc = geonamescache.GeonamesCache()

unidecoded_cities = pd.DataFrame(
    (unidecode(c["name"]) for c in gc.get_cities().values()), columns=["city"]
)
unidecoded_cities.describe()

,city
count,24336
unique,23022
top,Springfield
freq,8


Here we collect all of the cities availabe in `geonamescache` into a `pandas.DataFrame`. We normalize the city names by applying both `unidecode` and lowercasing. The original city names can be dropped since we will not be needing them for merging.

In [5]:
all_cities = pd.DataFrame(
    gc.get_cities().values(),
    columns=["name", "latitude", "longitude", "countrycode", "population"],
)
all_cities.insert(
    0,
    "city_lower",
    all_cities.name.apply(unidecode).apply(str.lower),
    allow_duplicates=True,
)
all_cities.drop(["name"], axis=1, inplace=True, errors="ignore")
all_cities

,city_lower,latitude,longitude,countrycode,population
0,andorra la vella,42.50779,1.52109,AD,20430
1,umm al quwain city,25.56473,55.55517,AE,62747
2,ras al khaimah city,25.78953,55.94320,AE,351943
3,zayed city,23.65416,53.70522,AE,63482
4,khawr fakkan,25.33132,56.34199,AE,40677
...,...,...,...,...,...
24331,bulawayo,-20.15000,28.58333,ZW,699385
24332,bindura,-17.30192,31.33056,ZW,37423
24333,beitbridge,-22.21667,30.00000,ZW,26459
24334,epworth,-17.89000,31.14750,ZW,123250


We can group by the city name, and filter out all the groups that only have one city in them. These are the cities with unique names. These can be concatenated back into a single `pandas.DataFrame`.

In [6]:
unique_cities = pd.concat(
    group_df for _, group_df in all_cities.groupby("city_lower") if len(group_df) == 1
)
unique_cities

,city_lower,latitude,longitude,countrycode,population
17194,'abasan al kabirah,31.31913,34.34005,PS,18163
11841,'afak,32.06430,45.24743,IQ,21888
19168,'afrin,36.51194,36.86954,SY,48693
5885,'ain abid,36.23194,6.94333,DZ,29486
5882,'ain benian,36.80277,2.92185,DZ,31102
...,...,...,...,...,...
17596,zyablikovo,55.61667,37.76667,RU,129000
16828,zyrardow,52.04880,20.44599,PL,41179
13756,zyryanovsk,49.72654,84.27318,KZ,44929
17595,zyuzino,55.65608,37.56846,RU,121000


We do the same group by operation on city name, but this time we look for groups that have more than one city in them. These are the non-unique city names. For these groups we need to choose the city with the highest population.

In [7]:
non_unique_cities = (
    group_df for _, group_df in all_cities.groupby("city_lower") if len(group_df) > 1
)


def get_highest_population_city(df: pd.DataFrame) -> pd.DataFrame:
    df.reset_index(drop=True, inplace=True)
    return df.iloc[df.population.idxmax()]


highest_population_non_unique_cities = pd.DataFrame(
    get_highest_population_city(group_df) for group_df in non_unique_cities
)
highest_population_non_unique_cities

,city_lower,latitude,longitude,countrycode,population
0,abadan,30.33920,48.30430,IR,370180
0,aberdeen,57.14369,-2.09814,GB,196670
0,acton,51.50901,-0.27620,GB,53689
1,addison,41.93170,-87.98896,US,37208
0,adra,36.74961,-3.02055,ES,24373
...,...,...,...,...,...
0,zarechnyy,53.20356,45.19227,RU,63579
0,zhaoyuan,37.36497,120.40997,CN,120000
0,zheleznogorsk,52.33100,35.37110,RU,97900
1,zhicheng,30.29556,111.50472,CN,159383


Now we can concatenate both of the above `DataFrame`s together to get a `DataFrame` that contains all unique cities from `geonamescache`. We drop the population column because we no longer need it.

In [8]:
all_unique_cities = pd.concat([unique_cities, highest_population_non_unique_cities])
all_unique_cities.set_index("city_lower", drop=True, inplace=True)
all_unique_cities.drop("population", inplace=True, axis=1, errors="ignore")
all_unique_cities

,latitude,longitude,countrycode
city_lower,,,
'abasan al kabirah,31.31913,34.34005,PS
'afak,32.06430,45.24743,IQ
'afrin,36.51194,36.86954,SY
'ain abid,36.23194,6.94333,DZ
'ain benian,36.80277,2.92185,DZ
...,...,...,...
zarechnyy,53.20356,45.19227,RU
zhaoyuan,37.36497,120.40997,CN
zheleznogorsk,52.33100,35.37110,RU


## Merging headlines and `geonamescache` DataFrames 

We're merging the two `DataFrames` on the `city_lower` column. Once that is done, we can drop the `city_lower` column.

In [9]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 200)
headlines_with_geo = pd.merge(parsed_headlines, all_unique_cities, on="city_lower", how="inner")
headlines_with_geo.drop("city_lower", axis=1, inplace=True, errors="ignore")
headlines_with_geo

,headline,city,latitude,longitude,countrycode
0,Case of Hepatitis A Reported in Calgary,Calgary,51.05011,-114.08529,CA
1,More people in Nadi are infected with HIV every year,Nadi,-17.80309,177.41617,FJ
2,Lower Hospitalization in Janesville after Varicella Vaccine becomes Mandatory,Janesville,42.68279,-89.01872,US
3,Contaminated Meat Brings Trouble for Bonn Farmers,Bonn,50.73438,7.09549,DE
4,Molo Cholera Spread Causing Concern,Molo,-0.24849,35.73194,KE
5,Bronchitis re-emerges in Tehran,Tehran,35.69439,51.42151,IR
6,Schools in Yaounde Closed Due to Mumps Outbreak,Yaounde,3.86667,11.51667,CM
7,Zika arrives in San Benito,San Benito,16.91675,-89.91898,GT
8,Lower Hospitalization in Monroe after Hepatitis D Vaccine becomes Mandatory,Monroe,32.50931,-92.11930,US
9,The Spread of Respiratory Syncytial Virus in Monroe has been Confirmed,Monroe,32.50931,-92.11930,US
